In [28]:
#imports the data set arrays. the arrays are the 2d array representation of the images, pre-processed to be grayscale and 180x180 pixels.

import numpy as np

nondemented_array = np.load(r"AugmentedAlzheimerDataset\NonDemented.npy", allow_pickle = True)
verymilddemented_array = np.load(r"AugmentedAlzheimerDataset\VeryMildDemented.npy", allow_pickle = True)
milddemented_array = np.load(r"AugmentedAlzheimerDataset\MildDemented.npy", allow_pickle = True)
moderatedemented_array = np.load(r"AugmentedAlzheimerDataset\ModerateDemented.npy", allow_pickle = True)

train_size = int(min(len(nondemented_array), len(verymilddemented_array), len(milddemented_array), len(moderatedemented_array)) / 2)
#the size of each sample size in the training batch is half of the minimum sample size


training_data = np.concatenate((nondemented_array[0: train_size], verymilddemented_array[0: train_size], milddemented_array[0: train_size], moderatedemented_array[0: train_size]))
testing_data = np.concatenate((nondemented_array[train_size:], verymilddemented_array[train_size:], milddemented_array[train_size:], moderatedemented_array[train_size:]))
#creates the training data by concatenating all of the arrays of length train_size together, creates testing data by concatenating the rest of the data


np.random.shuffle(training_data)
np.random.shuffle(testing_data)
#shuffles the training and testing data

In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)
        self.pool1 = nn.MaxPool2d((2, 2))
        self.pool2 = nn.MaxPool2d((2, 2))
        self.pool3 = nn.MaxPool2d((2, 2))
        self.fc1 = nn.Linear(46208, 46208) #size fo the fully connected layer was determined by the size of the pictures, which are 180 x 180 pixels
        self.fc2 = nn.Linear(46208, 2)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = self.pool3(x)
        x = x.flatten(start_dim=1) # flattening out
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)

net = Net()

In [29]:
optimizer = optim.Adam(net.parameters(), lr = 0.001)
loss_function = nn.MSELoss()

training_picture = torch.Tensor([i[0] for i in training_data]).view(-1, 180, 180)
training_picture = training_picture/255.0
training_class = torch.Tensor([i[1] for i in training_data])

testing_picture = torch.Tensor([i[0] for i in testing_data]).view(-1, 180, 180)
testing_picture = testing_picture/255.0
testing_class = torch.Tensor([i[1] for i in testing_data])


In [ ]:
BATCH_SIZE = 100
EPOCHS = 3

for epoch in range(EPOCHS):
    for i in (range(0, len(training_picture), BATCH_SIZE)):
        batch_pic = training_picture[i: i + BATCH_SIZE].view(-1, 1, 180, 180)
        batch_class = train_class[i: i + BATCH_SIZE]
        
        net.zero_grad()
        outputs = net(batch_pic)
        loss = loss_function(outputs, batch_class)
        loss.backward()
        optimizer.step()
        
print(loss)

In [ ]:
correct, total = 0, 0
with torch.no_grad():
    for i in (range(len(testing_picture))):
        real_class = torch.argmax(testing_class[i])
        net_out = net(testing_picture[i].view(-1, 1, 50, 50))[0]
        predicted_class = torch.argmax(net_out)
        if predicted_class == real_class:
            correct += 1
        total += 1
        
print(round(correct/total, 3))